# Azure聊天完成模型与您自己的数据（预览）
本示例展示了如何使用Azure OpenAI服务模型与您自己的数据。该功能目前处于预览阶段。

在您的数据上使用Azure OpenAI可以让您运行支持的聊天模型，如GPT-3.5-Turbo和GPT-4，而无需训练或微调模型。在您的数据上运行模型可以让您在更准确和更快的速度上进行聊天和分析您的数据。Azure OpenAI在您的数据上的一个关键优势是它能够定制会话AI的内容。因为模型可以访问并引用特定来源来支持其响应，答案不仅基于其预训练知识，还基于指定数据源中可用的最新信息。这些基础数据还有助于模型避免基于过时或不正确信息生成响应。

Azure OpenAI在您自己的数据上与Azure AI搜索（以前称为Azure认知搜索）提供了一个可定制的、预构建的知识检索解决方案，可以构建一个会话AI应用程序。要查看有关知识检索和语义搜索的替代方法，请查看[向量数据库](https://github.com/openai/openai-cookbook/tree/main/examples/vector_databases)的示例。

## 工作原理

[Azure OpenAI on your own data](https://learn.microsoft.com/azure/ai-services/openai/concepts/use-your-data) 将模型与您的数据连接起来，使其能够以增强模型输出的方式检索和利用数据。与 Azure AI Search 一起，根据用户输入和提供的对话历史，从指定的数据源中检索数据。然后，对数据进行增强处理，并重新提交为提示给模型，为模型提供可以用来生成响应的上下文信息。

有关更多信息，请参阅[Azure OpenAI 服务的数据、隐私和安全性](https://learn.microsoft.com/legal/cognitive-services/openai/data-privacy?context=%2Fazure%2Fai-services%2Fopenai%2Fcontext%2Fcontext)。

## 先决条件
为了开始，我们将介绍一些先决条件。

要正确访问Azure OpenAI服务，我们需要在[Azure门户](https://portal.azure.com)上创建适当的资源（您可以在[Microsoft Docs](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal)中查看如何执行此操作的详细指南）

要在Azure OpenAI模型中使用您自己的数据，您将需要：

1. Azure OpenAI访问权限以及部署了聊天模型的资源（例如，GPT-3或GPT-4）
2. Azure AI搜索（之前称为Azure Cognitive Search）资源
3. Azure Blob存储资源
4. 作为数据使用的文档（请参阅[数据源选项](https://learn.microsoft.com/azure/ai-services/openai/concepts/use-your-data#data-source-options))

要了解如何将您的文档上传到Blob存储并使用Azure AI Studio创建索引的完整步骤，请参阅此[快速入门](https://learn.microsoft.com/azure/ai-services/openai/use-your-data-quickstart?pivots=programming-language-studio&tabs=command-line)。

## 设置

首先，我们安装必要的依赖项。

In [ ]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install python-dotenv

在这个示例中，我们将使用`dotenv`来加载我们的环境变量。为了连接Azure OpenAI和搜索索引，以下变量应该以`KEY=VALUE`的格式添加到一个名为`.env`的文件中：

* `AZURE_OPENAI_ENDPOINT` - Azure OpenAI的终端。可以在Azure门户中的Azure OpenAI资源的“密钥和终端”下找到。
* `AZURE_OPENAI_API_KEY` - Azure OpenAI的API密钥。可以在Azure门户中的Azure OpenAI资源的“密钥和终端”下找到。如果使用Azure Active Directory身份验证，则可以省略（参见下面的“使用Microsoft Active Directory进行身份验证”）
* `SEARCH_ENDPOINT` - AI搜索的终端。这个URL可以在Azure门户上搜索资源的“概述”中找到。
* `SEARCH_KEY` - AI搜索的API密钥。可以在Azure门户上搜索资源的“密钥”下找到。
* `SEARCH_INDEX_NAME` - 您使用自己的数据创建的索引的名称。

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

### 认证

Azure OpenAI 服务支持多种认证机制，包括 API 密钥和 Azure Active Directory 令牌凭据。

In [2]:
use_azure_active_directory = False  # 将此标志设置为 True，如果您正在使用 Azure Active Directory。

#### 使用API密钥进行身份验证

要设置OpenAI SDK以使用*Azure API密钥*，我们需要将`api_key`设置为与您的端点关联的密钥（您可以在[Azure门户](https://portal.azure.com)的*"资源管理"*下的*"密钥和端点"*中找到此密钥）。您还将在此处找到您资源的端点。

In [3]:
if not use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]
    # 为我们要使用的模型设置部署名称
    deployment = "<deployment-id-of-the-model-to-use>"

    client = openai.AzureOpenAI(
        base_url=f"{endpoint}/openai/deployments/{deployment}/extensions",
        api_key=api_key,
        api_version="2023-09-01-preview"
    )

#### 使用Azure Active Directory进行身份验证
现在让我们看看如何通过Azure Active Directory进行身份验证。我们将从安装`azure-identity`库开始。这个库将提供我们需要进行身份验证的令牌凭据，并通过`get_bearer_token_provider`辅助函数帮助我们构建一个令牌凭据提供程序。建议使用`get_bearer_token_provider`而不是向`AzureOpenAI`提供静态令牌，因为这个API会自动为您缓存和刷新令牌。

有关如何设置Azure Active Directory身份验证与Azure OpenAI的更多信息，请参阅[文档](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity)。

In [ ]:
! pip install "azure-identity>=1.15.0"

In [5]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

if use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]
    # 为我们要使用的模型设置部署名称
    deployment = "<deployment-id-of-the-model-to-use>"

    client = openai.AzureOpenAI(
        base_url=f"{endpoint}/openai/deployments/{deployment}/extensions",
        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
        api_version="2023-09-01-preview"
    )

> 注意：如果未提供以下参数，则AzureOpenAI将从其对应的环境变量中推断出来：

- `api_key` 从 `AZURE_OPENAI_API_KEY`
- `azure_ad_token` 从 `AZURE_OPENAI_AD_TOKEN`
- `api_version` 从 `OPENAI_API_VERSION`
- `azure_endpoint` 从 `AZURE_OPENAI_ENDPOINT`

## 使用自己的数据完成聊天补全模型

### 设置上下文

在这个例子中，我们希望我们的模型基于Azure AI服务文档数据来生成回复。根据之前分享的[快速入门](https://learn.microsoft.com/azure/ai-services/openai/use-your-data-quickstart?tabs=command-line&pivots=programming-language-studio)，我们已将[markdown](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/ai-services/cognitive-services-and-machine-learning.md)文件添加到了我们的搜索索引中，该文件是关于[Azure AI服务和机器学习](https://learn.microsoft.com/azure/ai-services/cognitive-services-and-machine-learning)文档页面的内容。现在模型已经准备好回答关于Azure AI服务和机器学习的问题。

### 代码

现在我们可以在您自己的数据上使用Azure进行聊天完成。在`dataSources`中提供我们的搜索端点、密钥和索引名称，模型所提出的任何问题现在都将基于我们自己的数据。响应中还将提供一个名为`context`的额外属性，用于显示模型回答问题时参考的数据。

In [ ]:
completion = client.chat.completions.create(
    messages=[{"role": "user", "content": "What are the differences between Azure Machine Learning and Azure AI services?"}],
    model=deployment,
    extra_body={
        "dataSources": [
            {
                "type": "AzureCognitiveSearch",
                "parameters": {
                    "endpoint": os.environ["SEARCH_ENDPOINT"],
                    "key": os.environ["SEARCH_KEY"],
                    "indexName": os.environ["SEARCH_INDEX_NAME"],
                }
            }
        ]
    }
)
print(f"{completion.choices[0].message.role}: {completion.choices[0].message.content}")

# `context` 在 Azure 的 model_extra 中。
print(f"\nContext: {completion.choices[0].message.model_extra['context']['messages'][0]['content']}")

如果您希望从模型中流式传输响应，可以传递 `stream=True` 关键字参数：

In [ ]:
response = client.chat.completions.create(
    messages=[{"role": "user", "content": "What are the differences between Azure Machine Learning and Azure AI services?"}],
    model=deployment,
    extra_body={
        "dataSources": [
            {
                "type": "AzureCognitiveSearch",
                "parameters": {
                    "endpoint": os.environ["SEARCH_ENDPOINT"],
                    "key": os.environ["SEARCH_KEY"],
                    "indexName": os.environ["SEARCH_INDEX_NAME"],
                }
            }
        ]
    },
    stream=True,
)

for chunk in response:
    delta = chunk.choices[0].delta

    if delta.role:
        print("\n"+ delta.role + ": ", end="", flush=True)
    if delta.content:
        print(delta.content, end="", flush=True)
    if delta.model_extra.get("context"):
        print(f"Context: {delta.model_extra['context']}", end="", flush=True)